In [1]:
#Import das Libs
from cryptography.hazmat.primitives.keywrap import aes_key_wrap, aes_key_unwrap
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives.padding import PKCS7
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
import hashlib
import base64
import os

In [2]:
#Realiza-se o Upload do Arquivo que deseja-se Criptografar para o Google Colab.
path = "/content/Teste.txt"

In [3]:
#Criando uma Função para Gerar uma Chave + Salt
def derive_key(password: str, salt: bytes) -> bytes:

    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    return kdf.derive(password.encode())

In [4]:
#Criando uma Função para Encriptar um Arquivo
def encrypt_file(file_path: str, password: str) -> None:
    with open(file_path, 'rb') as f:
        plaintext = f.read()

    # Geração da Chave + Salt
    salt = os.urandom(16)
    key = derive_key(password, salt)

    # Start do Vetor de inicialização (IV)
    iv = os.urandom(16)

    # Setup do cifrador
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()

    # Aplicação do Padding e da Criptografia AWS
    padder = PKCS7(algorithms.AES.block_size).padder()
    padded_data = padder.update(plaintext) + padder.finalize()
    ciphertext = encryptor.update(padded_data) + encryptor.finalize()

    # Salvando uma cópia do Arquivo Encriptado com AES
    encrypted_file_path = file_path + '.enc'

    with open(encrypted_file_path, 'wb') as f:
        f.write(salt + iv + ciphertext)

    #Exibição do Resultado
    print(f"Arquivo criptografado salvo em: {encrypted_file_path}")

In [5]:
#Criando uma Função para Descriptar um Arquivo
def decrypt_file(file_path: str, password: str) -> None:
    with open(file_path, 'rb') as f:
        data = f.read()

    # Extração do SALT, IV + Texto Criptografado
    salt = data[:16]
    iv = data[16:32]
    ciphertext = data[32:]

    # Derivação da chave
    key = derive_key(password, salt)

    # Setup do Decifrador
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    decryptor = cipher.decryptor()

    # Aplicação da Descriptografia + Remoção do Padding
    padded_data = decryptor.update(ciphertext) + decryptor.finalize()
    unpadder = PKCS7(algorithms.AES.block_size).unpadder()
    plaintext = unpadder.update(padded_data) + unpadder.finalize()

    # Salvando uma Cópia do Arquivo Descriptografado
    decrypted_file_path = file_path.replace('.enc', '.dec')
    with open(decrypted_file_path, 'wb') as f:
        f.write(plaintext)

    #Exibição do Resultado
    print(f"Arquivo descriptografado salvo em: {decrypted_file_path}")

In [6]:
#Aplicação da Criptografia (Caminho do Arquivo , Senha)
encrypt_file(path, '1234')

Arquivo criptografado salvo em: /content/Teste.txt.enc


In [7]:
#Aplicação da Descriptografia (Caminho do Arquivo , Senha)
decrypt_file('/content/Teste.txt.enc', '1234')

Arquivo descriptografado salvo em: /content/Teste.txt.dec


In [8]:
#Exibindo o Valor do Conteúdo Original
with open('/content/Teste.txt', 'r', encoding='utf-8') as arquivo:
            org = arquivo.read()
print("Conteúdo Original: " + str(org))

Conteúdo Original: Conteúdo de Teste


In [9]:
#Exibindo o Valor do Conteúdo Original Criptografado (Encoding latin-1 Utilizado para Mostrar os Caracteres)
with open('/content/Teste.txt.enc', 'r', encoding='latin-1') as arquivo:
            enc = arquivo.read()
print("Conteúdo Criptografado: " + str(enc))

Conteúdo Criptografado: ÜöJº5û¨IIâe=xjø¦-$8?@à¨a^/ÞùæËÊ¸äùx^f­.~æ´ÜwÙv%=


In [10]:
#Exibindo o Valor do Conteúdo Descriptografado
with open('/content/Teste.txt.dec', 'r', encoding='utf-8') as arquivo:
            dec = arquivo.read()
print("Conteúdo Original: " + str(dec))

Conteúdo Original: Conteúdo de Teste


In [11]:
#Apresentação dos Resultados
print(org)
print(enc)
print(dec)

Conteúdo de Teste
ÜöJº5û¨IIâe=xjø¦-$8?@à¨a^/ÞùæËÊ¸äùx^f­.~æ´ÜwÙv%=
Conteúdo de Teste
